In [1]:
from sklearn import preprocessing
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
# df = pd.read_csv('Taipei_people.csv',)
df = pd.read_csv('Taipei_people.csv',header = 0)
# 檔案明用錯應是月不是週

In [3]:
# df.columns

In [4]:
# df.rename(columns={ df.columns[0]: "year" }, inplace=True)
# df.columns

In [5]:
# 合併年和週
new_df_index = pd.DataFrame({})
for i in range(len(df)):
    temp = '-'.join([str(df['年'][i]), str(df['月'][i])])
    new_df_index = new_df_index.append([temp], ignore_index=True)
new_df_index.columns = ['年_月']
df = pd.concat([new_df_index, df], axis=1)
del df['年']
del df['月']
df[-5:]

,年_月,total,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16+
116,2017-9,2682893,24790,29796,30992,29514,29710,28314,21962,20233,21333,22004,21897,21856,22469,22302,23401,23932,2288388
117,2017-10,2683202,24411,29677,30682,30078,28845,29082,22567,19851,21470,21885,21938,21987,22099,22463,23137,24072,2288958
118,2017-11,2683321,24423,29352,30752,30318,28222,29434,23202,19748,21399,21542,22235,22137,21925,22418,22888,24201,2289125
119,2017-12,2683257,24117,29281,30555,30737,27644,29910,23725,19460,21485,21310,22415,22159,21920,22325,22829,24193,2289192
120,2018-1,2682721,23996,29161,30340,30966,27185,30217,23947,19519,21416,21168,22525,22137,21940,22355,22575,24199,2289075


In [6]:
import matplotlib.pyplot as plt
from scipy import interpolate
import datetime
import time
%matplotlib inline

In [7]:
from datetime import datetime, date, timedelta
import calendar

def get_month_range(year, month):
    start_date = date(year, month, 1)
    _, days_in_month = calendar.monthrange(start_date.year, start_date.month)
    end_date = start_date + timedelta(days=days_in_month-1)
    return (end_date)

In [8]:
month = range(1, 13)
year = range(2008, 2019)

empty_df = []
for y in year:
    before_week = 0
    for m in month:
        datetime_date = get_month_range(y, m)
        n_week = datetime_date.isocalendar()[1]
        for i in range(1, 7):
            before_datetime = datetime_date + timedelta(days=-i)
            if n_week < before_datetime.isocalendar()[1]:
                n_week = before_datetime.isocalendar()[1]
        
        record_week = n_week - before_week
        before_week = n_week
        
        empty_df.append([y, m, record_week])
        
year_month_num_week = pd.DataFrame(empty_df, columns=('year', 'month', 'num_week'))

In [9]:
year_month_num_week[:5]

,year,month,num_week
0,2008,1,5
1,2008,2,4
2,2008,3,5
3,2008,4,4
4,2008,5,4


In [10]:
new_index = []
xx = 0
for i in range(len(df)):
    new_index.append(xx)
    xx += year_month_num_week.num_week[i]
index = pd.DataFrame({'index':new_index})

In [11]:
new_df = pd.concat([index, df], axis=1)
df_pop = new_df.pop('年_月')
new_df[:5]

,index,total,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16+
0,0,2629689,20881,22224,22332,23030,23384,25336,25322,30664,29237,27669,33864,34311,35778,34031,33572,32559,2175495
1,5,2630966,21184,22261,22272,22890,23207,25402,24979,30334,29513,27569,33576,34430,35554,34145,33598,32604,2177448
2,9,2631605,21203,22263,22290,22772,23086,25162,24818,30086,29385,27808,33220,34306,35586,34274,33615,32706,2179025
3,14,2630966,21257,22301,22131,22762,22879,25042,24722,29731,29426,28079,32821,34232,35648,34194,33683,32772,2179286
4,18,2630515,21188,22326,22020,22871,22821,24746,24531,29393,29411,28382,32455,34116,35626,34221,33602,32939,2179867


In [12]:
new_df[-5:]

,index,total,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16+
116,505,2682893,24790,29796,30992,29514,29710,28314,21962,20233,21333,22004,21897,21856,22469,22302,23401,23932,2288388
117,509,2683202,24411,29677,30682,30078,28845,29082,22567,19851,21470,21885,21938,21987,22099,22463,23137,24072,2288958
118,514,2683321,24423,29352,30752,30318,28222,29434,23202,19748,21399,21542,22235,22137,21925,22418,22888,24201,2289125
119,518,2683257,24117,29281,30555,30737,27644,29910,23725,19460,21485,21310,22415,22159,21920,22325,22829,24193,2289192
120,522,2682721,23996,29161,30340,30966,27185,30217,23947,19519,21416,21168,22525,22137,21940,22355,22575,24199,2289075


In [13]:
new_df = new_df.set_index(new_df.pop('index'))
new_df = new_df.reindex(np.arange(new_df.index.min(), new_df.index.max()+1))
new_df = new_df.interpolate()  #內插

In [14]:
new_df[:10]

,total,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16+
index,,,,,,,,,,,,,,,,,,
0,2629689.00,20881.00,22224.0,22332.0,23030.0,23384.00,25336.0,25322.00,30664.0,29237.0,27669.00,33864.0,34311.0,35778.0,34031.00,33572.00,32559.0,2175495.00
1,2629944.40,20941.60,22231.4,22320.0,23002.0,23348.60,25349.2,25253.40,30598.0,29292.2,27649.00,33806.4,34334.8,35733.2,34053.80,33577.20,32568.0,2175885.60
2,2630199.80,21002.20,22238.8,22308.0,22974.0,23313.20,25362.4,25184.80,30532.0,29347.4,27629.00,33748.8,34358.6,35688.4,34076.60,33582.40,32577.0,2176276.20
3,2630455.20,21062.80,22246.2,22296.0,22946.0,23277.80,25375.6,25116.20,30466.0,29402.6,27609.00,33691.2,34382.4,35643.6,34099.40,33587.60,32586.0,2176666.80
4,2630710.60,21123.40,22253.6,22284.0,22918.0,23242.40,25388.8,25047.60,30400.0,29457.8,27589.00,33633.6,34406.2,35598.8,34122.20,33592.80,32595.0,2177057.40
5,2630966.00,21184.00,22261.0,22272.0,22890.0,23207.00,25402.0,24979.00,30334.0,29513.0,27569.00,33576.0,34430.0,35554.0,34145.00,33598.00,32604.0,2177448.00
6,2631125.75,21188.75,22261.5,22276.5,22860.5,23176.75,25342.0,24938.75,30272.0,29481.0,27628.75,33487.0,34399.0,35562.0,34177.25,33602.25,32629.5,2177842.25
7,2631285.50,21193.50,22262.0,22281.0,22831.0,23146.50,25282.0,24898.50,30210.0,29449.0,27688.50,33398.0,34368.0,35570.0,34209.50,33606.50,32655.0,2178236.50
8,2631445.25,21198.25,22262.5,22285.5,22801.5,23116.25,25222.0,24858.25,30148.0,29417.0,27748.25,33309.0,34337.0,35578.0,34241.75,33610.75,32680.5,2178630.75


In [15]:
new_df[-10:]

,total,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16+
index,,,,,,,,,,,,,,,,,,
513,2683297.2,24420.60,29417.00,30738.00,30270.00,28346.60,29363.60,23075.00,19768.60,21413.20,21610.6,22175.6,22107.0,21959.80,22427.00,22937.80,24175.2,2289091.60
514,2683321.0,24423.00,29352.00,30752.00,30318.00,28222.00,29434.00,23202.00,19748.00,21399.00,21542.0,22235.0,22137.0,21925.00,22418.00,22888.00,24201.0,2289125.00
515,2683305.0,24346.50,29334.25,30702.75,30422.75,28077.50,29553.00,23332.75,19676.00,21420.50,21484.0,22280.0,22142.5,21923.75,22394.75,22873.25,24199.0,2289141.75
516,2683289.0,24270.00,29316.50,30653.50,30527.50,27933.00,29672.00,23463.50,19604.00,21442.00,21426.0,22325.0,22148.0,21922.50,22371.50,22858.50,24197.0,2289158.50
517,2683273.0,24193.50,29298.75,30604.25,30632.25,27788.50,29791.00,23594.25,19532.00,21463.50,21368.0,22370.0,22153.5,21921.25,22348.25,22843.75,24195.0,2289175.25
518,2683257.0,24117.00,29281.00,30555.00,30737.00,27644.00,29910.00,23725.00,19460.00,21485.00,21310.0,22415.0,22159.0,21920.00,22325.00,22829.00,24193.0,2289192.00
519,2683123.0,24086.75,29251.00,30501.25,30794.25,27529.25,29986.75,23780.50,19474.75,21467.75,21274.5,22442.5,22153.5,21925.00,22332.50,22765.50,24194.5,2289162.75
520,2682989.0,24056.50,29221.00,30447.50,30851.50,27414.50,30063.50,23836.00,19489.50,21450.50,21239.0,22470.0,22148.0,21930.00,22340.00,22702.00,24196.0,2289133.50
521,2682855.0,24026.25,29191.00,30393.75,30908.75,27299.75,30140.25,23891.50,19504.25,21433.25,21203.5,22497.5,22142.5,21935.00,22347.50,22638.50,24197.5,2289104.25


In [16]:
totalweeks_year = year_month_num_week.groupby(['year'], as_index=False)['num_week'].sum()

In [17]:
year = []
num_week = []

for i in range(len(totalweeks_year.year)):
    year_ = totalweeks_year.year[i]
    num = totalweeks_year.num_week[i]
    for j in range(1, num+1):
        year.append(year_)
        num_week.append(j)
        
new_index = pd.DataFrame({'	year' : year, 'week' : num_week})

In [18]:
X = pd.concat([new_index, new_df], axis=1)
X = X.dropna()

In [19]:
X[:5]

,year,week,total,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16+
0,2008,1,2629689.0,20881.0,22224.0,22332.0,23030.0,23384.0,25336.0,25322.0,30664.0,29237.0,27669.0,33864.0,34311.0,35778.0,34031.0,33572.0,32559.0,2175495.0
1,2008,2,2629944.4,20941.6,22231.4,22320.0,23002.0,23348.6,25349.2,25253.4,30598.0,29292.2,27649.0,33806.4,34334.8,35733.2,34053.8,33577.2,32568.0,2175885.6
2,2008,3,2630199.8,21002.2,22238.8,22308.0,22974.0,23313.2,25362.4,25184.8,30532.0,29347.4,27629.0,33748.8,34358.6,35688.4,34076.6,33582.4,32577.0,2176276.2
3,2008,4,2630455.2,21062.8,22246.2,22296.0,22946.0,23277.8,25375.6,25116.2,30466.0,29402.6,27609.0,33691.2,34382.4,35643.6,34099.4,33587.6,32586.0,2176666.8
4,2008,5,2630710.6,21123.4,22253.6,22284.0,22918.0,23242.4,25388.8,25047.6,30400.0,29457.8,27589.0,33633.6,34406.2,35598.8,34122.2,33592.8,32595.0,2177057.4


In [20]:
X=X[:522]
X['0~15'] = X['0']+X['1']+X['2']+X['3']+X['4']+X['5']+X['6']+X['7']+X['8']+X['9']+X['10']+X['11']+X['12']+X['13']+X['14']+X['15']

In [21]:
X.head()

,year,week,total,0,1,2,3,4,5,6,...,8,9,10,11,12,13,14,15,16+,0~15
0,2008,1,2629689.0,20881.0,22224.0,22332.0,23030.0,23384.0,25336.0,25322.0,...,29237.0,27669.0,33864.0,34311.0,35778.0,34031.0,33572.0,32559.0,2175495.0,454194.0
1,2008,2,2629944.4,20941.6,22231.4,22320.0,23002.0,23348.6,25349.2,25253.4,...,29292.2,27649.0,33806.4,34334.8,35733.2,34053.8,33577.2,32568.0,2175885.6,454058.8
2,2008,3,2630199.8,21002.2,22238.8,22308.0,22974.0,23313.2,25362.4,25184.8,...,29347.4,27629.0,33748.8,34358.6,35688.4,34076.6,33582.4,32577.0,2176276.2,453923.6
3,2008,4,2630455.2,21062.8,22246.2,22296.0,22946.0,23277.8,25375.6,25116.2,...,29402.6,27609.0,33691.2,34382.4,35643.6,34099.4,33587.6,32586.0,2176666.8,453788.4
4,2008,5,2630710.6,21123.4,22253.6,22284.0,22918.0,23242.4,25388.8,25047.6,...,29457.8,27589.0,33633.6,34406.2,35598.8,34122.2,33592.8,32595.0,2177057.4,453653.2


In [22]:
X.tail()

,year,week,total,0,1,2,3,4,5,6,...,8,9,10,11,12,13,14,15,16+,0~15
517,2017,48,2683273.0,24193.50,29298.75,30604.25,30632.25,27788.50,29791.00,23594.25,...,21463.50,21368.0,22370.0,22153.5,21921.25,22348.25,22843.75,24195.0,2289175.25,394097.75
518,2017,49,2683257.0,24117.00,29281.00,30555.00,30737.00,27644.00,29910.00,23725.00,...,21485.00,21310.0,22415.0,22159.0,21920.00,22325.00,22829.00,24193.0,2289192.00,394065.00
519,2017,50,2683123.0,24086.75,29251.00,30501.25,30794.25,27529.25,29986.75,23780.50,...,21467.75,21274.5,22442.5,22153.5,21925.00,22332.50,22765.50,24194.5,2289162.75,393960.25
520,2017,51,2682989.0,24056.50,29221.00,30447.50,30851.50,27414.50,30063.50,23836.00,...,21450.50,21239.0,22470.0,22148.0,21930.00,22340.00,22702.00,24196.0,2289133.50,393855.50
521,2017,52,2682855.0,24026.25,29191.00,30393.75,30908.75,27299.75,30140.25,23891.50,...,21433.25,21203.5,22497.5,22142.5,21935.00,22347.50,22638.50,24197.5,2289104.25,393750.75


In [23]:
X=round(X)  # 四捨五入

In [24]:
# X.to_csv('Taipei_people_use.csv',index=False)